In [1]:
import json

from pathlib import Path

from kptncook.config import settings
from kptncook.models import Recipe
from kptncook.mealie import kptncook_to_mealie

from kptncook.mealie import MealieApiClient

In [2]:
example_path = Path.cwd().parent / "tests" / "fixtures" / "kptncook_example.json"
with example_path.open("r") as f:
    example = json.load(f)
recipe = Recipe.parse_obj(example)

In [3]:
mealie_recipe = kptncook_to_mealie(recipe)

In [4]:
# mealie_recipe.image_url

# Import via Api

In [5]:
client = MealieApiClient("http://localhost:9000/api")
client.login()

 ········


In [6]:
%%time
client.create_recipe(mealie_recipe)

CPU times: user 30.9 ms, sys: 4.49 ms, total: 35.4 ms
Wall time: 439 ms


Recipe(id=UUID('b32f7125-b9d2-43ee-957d-53c4a15f9f10'), user_id=UUID('899979e8-f240-4d53-8ed3-eaeac21ecd53'), group_id=UUID('71aae446-1aa5-4db7-9782-4021692cd0ce'), name='Chili sin Carne', slug='chili-sin-carne', image=None, recipe_yield=None, total_time=None, prep_time=None, cook_time=None, perform_time=None, description='', recipe_category=[], tags=[], tools=[], rating=None, org_url=None, recipe_ingredient=[], date_added=None, date_updated=None, recipe_instructions=[], nutrition=Nutrition(calories='835', fat_content=None, protein_content=None, carbohydrate_content=None, fiber_content=None, sodium_content=None, sugar_content=None), settings=RecipeSettings(public=False, show_nutrition=False, show_assets=False, landscape_view=False, disable_comments=True, disable_amount=True, locked=False), assets=[], notes=[RecipeNote(title='author comment', text='Hot stuff')], extras={})